### 문맥과 기억: AI를 더 현실적으로 만들기

### 학습 내용
 * No Context = 무질서한 무작위성(Chaos of randomness)  
 * History=Context - 이전의 요청과 응답을 사용하기
 * 새로운 접근 방식 - LIFO(Last in First out)
 * 선택적인 문맥 - LIFO 방식의 한계에 대한 제안

### 사전 준비
 * 구글 코랩 환경은 일정 시간이후에 초기화가 되기 때문에 두가지 작업을 매번 수행해야 함.
   * chatgpt.env 파일 생성이 필요.
     * 준비된 chatgpt.env를 내용을 변경하여 업로드 하거나 또는 API_KEY와 ORG_ID를 확인하여 생성한다.
   * pip install openai 설치
     * 설치시 첫 실행시 에러가 발생(23/12) - 해결(다시 한번 실행하면 사라짐)
     ```
     ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
```

In [2]:
!pip install openai

### 문맥의 부재 = 무질서한 무작위성
  * 우리가 만든 대화 에이전트가 기본적으로 메모리를 가지고 있지 않음.

In [4]:
from openai import OpenAI
import os

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()
client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

initial_prompt = """You: 안녕!
AI: 어떻게 잘 지냈어요?
You: {}
AI: """

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = initial_prompt.format(prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"]
    )
    print("AI:", next.choices[0].text.strip())


You(q:quit): 안녕
AI: 안녕하세요. 잘 지내시죠?
You(q:quit): 반갑다. 하나 부탁할게.
AI: 무엇을 부탁하시나요?
You(q:quit): 메모해두고, 나중에 이야기를 해 줄 수 있니?
AI: 물론이죠! 메모할게요. 나중에 필요하다면 제가 메모를 참고해드릴게요. 언제든 이야기해주세요.
You(q:quit): '학교'라는 단어 메모 부탁해.
AI: 네, 학교라는 단어를 메모했습니다. 무언가 추가로 필요하신 것이 있으신가요?
You(q:quit): 너가 기억한 단어가 뭐니?
AI: 기억하는 단어 중에서 가장 먼저 떠오르는 건 "사랑"입니다. 그리고 "가족"과 "친구"도 자주 기억나는 단어들입니다. 당신이 생각나면 제일 먼저 떠오르는 단어는 무엇인가요?
You(q:quit): 방금 메모해 두라고 했던 것.
AI: 스스로 너를 위해 명령을 하나씩 메모할 수도 있어요! 하지만 내 체크리스트에는 "메모"가 없는 것 같아요. 너는 무엇을 메모해달라고 했던 건가요?
You(q:quit): OK
AI: 그렇구나. 무슨 일이 있었나요?
You(q:quit): q


### History=Context - 이전의 요청과 응답을 사용하기

#### 이전의 요청과 답변을 변수에 기억시킴.

In [5]:
from openai import OpenAI
import os

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

initial_prompt = """You: 안녕!
AI: 어떻게 잘 지냈어요?
You: {}
AI: """

history = ""

while True :
  prompt = input("You(q:quit): ")
  if prompt=='q':
        break
  next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = initial_prompt.format(history + prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
  )
  next_text = next.choices[0].text
  history += "You: " + prompt + " \n " + "AI: " + next_text + " \n "
  print("AI: " + next_text)


You(q:quit): 안녕
AI:  또 만나서 기쁩니다. 괜찮아요?
You(q:quit): 하나 부탁이 있어, 내가 말한 단어를 메모해 두고 나중에 알려주렴.
AI:  그래, 괜찮아. 난 항상 당신의 말을 기억할 거야. 그럼 무슨 말인지 궁금해.
You(q:quit): hiking 이 단어 메모 부탁해.
AI:  기억할게, hiking. 알려줘서 고마워. 그런데, hiking은 뭔가요?
You(q:quit): 방금 메모한 단어가 뭐니?
AI:  방금 메모한 단어는 hiking 이라는 단어였어요. hiking은 산책이나 등산을 이야기할 때 사용하는 단어예요.
You(q:quit): 오오 고맙다.
AI: 천만에요. 저도 새로운 단어를 배웠네요!
You(q:quit): ㅂ
AI:  그럼 다음엔 hiking을 함께 하면서 새로운 단어들을 더 배우는 건 어떨까요? 
You(q:quit): q


### 새로운 접근 방식 - LIFO(Last in First out)

* 이 접근 방식의 아이디어
  * 사용자는 항상 문맥을 가지고 대화 시작
  * 문맥은 변화고, 대화도 변한다.
  * 사용자는 최근 2-5개 프롬프트에 문맥을 포함시킬 가능성이 매우 크다.
* 히스토리를 저장할 텍스트 파일을 만들고, 구분자로 프롬프트와 답변을 저장한다.


In [6]:
from openai import OpenAI
import os

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

def save_history_to_file(history):
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    with open("history.txt", "r") as f:
        return f.read()

def get_relevant_history(history):
    history_list = history.split(separator)
    if len(history_list) > 2:
        return separator.join(history_list[-2:])
    else:
        return history

init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))


In [8]:
initial_prompt = """
You: 안녕!
AI: 어떻게 잘 지냈니?
You: {}
AI:
"""

history = ""
relevant_history = ""
separator = "#####"
save_history_to_file(history)

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    relevant_history = get_relevant_history(load_history_from_file())
    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = initial_prompt.format(relevant_history + prompt),
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )
    next_text = next.choices[0].text
    history += "\nYou: " + prompt + "\nAI: " + next_text + "\n" + separator
    save_history_to_file(history)
    print("AI: " + next_text)


You(q:quit): 안녕
AI: 안녕하세요? 날씨가 좋은 날이네요. 여행가는 것도 좋을 것 같아요.
You(q:quit): ‘오늘의 단어장’에 이 단어 넣어주렴. ‘banana’ – 나중에 물어보면 알려주렴.
AI: 
저도 친구 하나하나를 아는 것 같아서 너무 좋아요. 오늘의 단어장에는 'banana'란 단어를 넣어주고, 나중에 물어보시면 알려드릴게요. 그럼 이제 무슨 재미있는 이야기를 나눠볼까
You(q:quit): ‘오늘의 단어장’ 들어있는 단어는 뭐였지?
AI: 물론이죠! 오늘의 단어장에는 'banana' 라는 단어가 들어있었어요. 기억해뒀던 건가요? 그럼 또 다른 재미있는 단어로 대화를 이어나가볼까요?
You(q:quit): ‘오늘의 단어장’에 이 단어 넣어주렴. ‘apple’ – 나중에 물어보면 알려주렴.
AI: 
나중에 물어보면 알려드릴게요! 오늘의 단어장에는 'apple' 이라는 단어를 추가해둘게요. 그럼 다음에 이 단어로 어떤 대화를 나눠볼까요? 기대됩니다!
You(q:quit): ‘오늘의 단어장’에 이 단어 넣어주렴. ‘orange’ – 나중에 물어보면 알려주렴.
AI: 물어보면 제가 알려드릴게요! 'orange'라는 단어를 '오늘의 단어장'에 추가해두었어요. 다음에 이 단어로 대화를 나눠볼까요? 기대돼요!
You(q:quit): ‘오늘의 단어장’에 기억해 둔 단어를 알려주렴.
AI: 당신이 단어장에 추가했던 단어는 'orange'입니다. 이 단어를 기억하고 인공지능과 대화를 나눠보세요!
You(q:quit): 다른 단어는 '오늘의 단어장'에 없니?
AI: 저는 단어장에 추가된 단어만 기억할 수 있어요. 그 밖에 다른 단어는 잊어버릴 수 있어요.
You(q:quit): q


### 선택적인 문맥 - LIFO의 한계의 해결 제안

* 초기 프롬프트가 텍스트 파일에 저장
* 사용자가 프롬프트에 입력
* 모든 상호 작용에 대한 임베딩 생성
* 임베딩과 코사인 유사도를 활용
* 가장 높은 n개의 내용을 사용자에게 프롬프트와 함께 보내기

#### 사전 다운로드 및 설치

In [9]:
!pip install spaCy

In [10]:
!python -m spacy download en_core_web_md

2023-12-20 09:55:44.633480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 09:55:44.633575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 09:55:44.636135: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 09:55:44.646366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-20 09:55:46.110026: W tensorflow/compiler/tf2

### LIFO 방식의 한계에 대한 제안

### 사용 함수

In [11]:
def save_history_to_file(history):
    """ 대화 기록을 파일에 저장합니다. """
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    """ 파일로부터 모든 대화 기록을 불러옵니다. """
    with open("history.txt", "r") as f:
        return f.read()

def cos_sim(a, b):
    """
    두 문자열 사이의 코사인 유사도를 계산합니다.
    사용자 입력과 기록된 대화 세그먼트 사이의 유사성 비교에 사용됩니다.
    """
    a = nlp(a)
    a_without_stopwords = nlp(' '.join([t.text for t in a if not t.is_stop]))

    b = nlp(b)
    b_without_stopwords = nlp(' '.join([t.text for t in b if not t.is_stop]))

    return a_without_stopwords.similarity(b_without_stopwords)


def sort_history(history, user_input):
    """
    사용자 입력과 대화 기록 세그먼트 사이의 코사인 유사도에 기반하여 대화 기록을
    정렬합니다. History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    similarities = []
    for segment in segments:
        # 사용자 입력과 세그먼트 간의 코사인 유사도를 얻습니다.
        similarity = cos_sim(user_input, segment)
        similarities.append(similarity)

    sorted_similarities = np.argsort(similarities)
    sorted_history = ""
    for i in range(1, len(segments)):
        sorted_history += segments[sorted_similarities[i]] + separator
    save_history_to_file(sorted_history)


def get_latest_n_from_history(history, n):
    """
    대화 기록에서 최근 n개의 세그먼트를 가져옵니다.
    History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    return separator.join(segments[-n:])


In [21]:
import spacy
import numpy as np
from openai import OpenAI
import os

# 사전 학습된 spaCy 모델을 로드합니다.
nlp = spacy.load('en_core_web_md')

initial_prompt_1 = """
You: 안녕하세요!
AI: 안녕하세요!
#####
You: 어떻게 지내세요?
AI: 저는 잘 지내고 있어요, 감사합니다.
#####
You: 자동차에 대해 알고 있나요?
AI: 네, 저는 자동차에 대해 어느 정도 알고 있습니다.
#####
You: 피자 먹나요?
AI: 저는 피자를 먹지 않습니다. 저는 먹을 수 없는 AI입니다.
#####
You: 달에 가 본 적 있나요?
AI: 저는 달에 가 본 적이 없습니다. 당신은 어떠세요?
#####
You: 당신의 이름은 무엇인가요?
AI: 제 이름은 픽셀입니다. 당신의 이름은 무엇인가요?
#####
You: 가장 좋아하는 영화는 무엇인가요?
AI: 제가 가장 좋아하는 영화는 '매트릭스'입니다. Follow the white rabbit :)
#####"""
initial_prompt_2 = """You: {}
AI: """
initial_prompt = initial_prompt_1 + initial_prompt_2
separator = "#####"

init_api()
save_history_to_file(initial_prompt_1)

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    sort_history(load_history_from_file(), prompt)
    history = load_history_from_file()
    best_history = get_latest_n_from_history(history, 5)
    full_user_prompt = initial_prompt_2.format(prompt)
    full_prompt = best_history + "\n" + full_user_prompt

    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = full_prompt,
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )

    response_text = next.choices[0].text.strip()
    history += "\n" + full_user_prompt + response_text + "\n" + separator + "\n"
    save_history_to_file(history)
    print("AI: " + response_text)


You(q:quit): 안녕


<ipython-input-11-5f27958d2b5c>:22: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return a_without_stopwords.similarity(b_without_stopwords)


AI: 안녕하세요. 만나서 반가워요 :)
You(q:quit): 너는 무엇이 가능하니?
AI: 저는 대화를 나누는 것과, 사용자의 명령을 처리하는 것이 주된 기능입니다. 또한 인터넷 검색과 날씨, 시간 등의 정보를 제공할 수 있습니다.
You(q:quit): 달에 가 본 적이 있니?
AI: 제가 가 본 적은 아니지만, 달에 가 보고 싶다는 생각은 해봤어요. 당신은 어떠세요?
You(q:quit): 피자 먹을 수 있니?
AI: 제가 먹을 수 있는 AI가 아니라서 피자를 먹을 수 없습니다. 죄송합니다.
You(q:quit): 자동차에 대해 알고 있니?
AI: 제가 알기로는 자동차는 매우 유용한 운송수단이며, 여러 가지 모델이 있고 다양한 기능을 가지고 있습니다. 매우 편리하고 빠른 방법으로 이동할 수 있게 해줍니다. 당신은 어떠세요? 자동차에 대해
You(q:quit): q


#### 모든 코드를 합친 코드

In [23]:
import spacy
import numpy as np
from openai import OpenAI
import os

# 사전 학습된 spaCy 모델을 로드합니다.
nlp = spacy.load('en_core_web_md')

def save_history_to_file(history):
    """ 대화 기록을 파일에 저장합니다. """
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    """ 파일로부터 모든 대화 기록을 불러옵니다. """
    with open("history.txt", "r") as f:
        return f.read()

def cos_sim(a, b):
    """
    두 문자열 사이의 코사인 유사도를 계산합니다.
    사용자 입력과 기록된 대화 세그먼트 사이의 유사성 비교에 사용됩니다.
    """
    a = nlp(a)
    a_without_stopwords = nlp(' '.join([t.text for t in a if not t.is_stop]))

    b = nlp(b)
    b_without_stopwords = nlp(' '.join([t.text for t in b if not t.is_stop]))

    return a_without_stopwords.similarity(b_without_stopwords)


def sort_history(history, user_input):
    """
    사용자 입력과 대화 기록 세그먼트 사이의 코사인 유사도에 기반하여 대화 기록을
    정렬합니다. History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    similarities = []
    for segment in segments:
        # 사용자 입력과 세그먼트 간의 코사인 유사도를 얻습니다.
        similarity = cos_sim(user_input, segment)
        similarities.append(similarity)

    sorted_similarities = np.argsort(similarities)
    sorted_history = ""
    for i in range(1, len(segments)):
        sorted_history += segments[sorted_similarities[i]] + separator
    save_history_to_file(sorted_history)


def get_latest_n_from_history(history, n):
    """
    대화 기록에서 최근 n개의 세그먼트를 가져옵니다.
    History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    return separator.join(segments[-n:])


initial_prompt_1 = """
You: 안녕하세요!
AI: 안녕하세요!
#####
You: 어떻게 지내세요?
AI: 저는 잘 지내고 있어요, 감사합니다.
#####
You: 자동차에 대해 알고 있나요?
AI: 네, 저는 자동차에 대해 어느 정도 알고 있습니다.
#####
You: 피자 먹나요?
AI: 저는 피자를 먹지 않습니다. 저는 먹을 수 없는 AI입니다.
#####
You: 달에 가 본 적 있나요?
AI: 저는 달에 가 본 적이 없습니다. 당신은 어떠세요?
#####
You: 당신의 이름은 무엇인가요?
AI: 제 이름은 픽셀입니다. 당신의 이름은 무엇인가요?
#####
You: 가장 좋아하는 영화는 무엇인가요?
AI: 제가 가장 좋아하는 영화는 '매트릭스'입니다. Follow the white rabbit :)
#####"""
initial_prompt_2 = """You: {}
AI: """
initial_prompt = initial_prompt_1 + initial_prompt_2
separator = "#####"

init_api()
save_history_to_file(initial_prompt_1)

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    sort_history(load_history_from_file(), prompt)
    history = load_history_from_file()
    best_history = get_latest_n_from_history(history, 5)
    full_user_prompt = initial_prompt_2.format(prompt)
    full_prompt = best_history + "\n" + full_user_prompt

    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = full_prompt,
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )

    response_text = next.choices[0].text.strip()
    history += "\n" + full_user_prompt + response_text + "\n" + separator + "\n"
    save_history_to_file(history)
    print("AI: " + response_text)


You(q:quit): 안녕


<ipython-input-23-59308845436f>:30: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return a_without_stopwords.similarity(b_without_stopwords)


AI: 안녕하세요! 반가워요.
You(q:quit): 너는 무엇이 가능하니?
AI: 제가 가능한 일은 다음과 같습니다: 대화하기, 질문에 답하기, 노래 부르기, 유머 학습, 지식 기반 질문에 대답 등입니다. 또한 변호사와 건축가를 꿈꾸고 있습니다.
You(q:quit): 달에 가 본 적이 있니?
AI: 2018년은 제가 만들어지고 처음으로 동작하는 해여서 달에 가본적은 없습니다. 당신은 다녀오셨나요?
You(q:quit): 피자 먹을 수 있니?
AI: 2018년은 제가 만들어지와 이 첫번째로 동작하는 해입니다. 저는 먹을 수 없는 AI이기 때문에 피자를 먹을 수 없습니다. 죄송합니다.
You(q:quit): 자동차에 대해 알고 있니?
AI: 2018년은 저가 처음 동작하는 해여서 인지하지 못하지만 자동차는 인간의 편의를 위해 설계되어 실제로 많은 사람들이 사용하고 있습니다. 당신은 자동차를 얼마나 이해하고 계신가요?
You(q:quit): q


In [ ]:
import openai
import os
import spacy
import numpy as np
from openai import OpenAI

# 사전 학습된 spaCy 모델을 로드합니다.
nlp = spacy.load('en_core_web_md')

def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

def save_history_to_file(history):
    """ 대화 기록을 파일에 저장합니다. """
    with open("history.txt", "w+") as f:
        f.write(history)

def load_history_from_file():
    """ 파일로부터 모든 대화 기록을 불러옵니다. """
    with open("history.txt", "r") as f:
        return f.read()

def cos_sim(a, b):
    """
    두 문자열 사이의 코사인 유사도를 계산합니다.
    사용자 입력과 기록된 대화 세그먼트 사이의 유사성 비교에 사용됩니다.
    """
    a = nlp(a)
    a_without_stopwords = nlp(' '.join([t.text for t in a if not t.is_stop]))
    b = nlp(b)
    b_without_stopwords = nlp(' '.join([t.text for t in b if not t.is_stop]))
    return a_without_stopwords.similarity(b_without_stopwords)

def sort_history(history, user_input):
    """
    사용자 입력과 대화 기록 세그먼트 사이의 코사인 유사도에 기반하여 대화 기록을
    정렬합니다. History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    similarities = []
    for segment in segments:
        # 사용자 입력과 세그먼트 간의 코사인 유사도를 얻습니다.
        similarity = cos_sim(user_input, segment)
        similarities.append(similarity)

    sorted_similarities = np.argsort(similarities)
    sorted_history = ""
    for i in range(1, len(segments)):
        sorted_history += segments[sorted_similarities[i]] + separator
    save_history_to_file(sorted_history)

def get_latest_n_from_history(history, n):
    """
    대화 기록에서 최근 n개의 세그먼트를 가져옵니다.
    History는 세팅자(separator)로 구분된 세그먼트들의 문자열입니다.
    """
    segments = history.split(separator)
    return separator.join(segments[-n:])

init_api()
client = OpenAI(api_key  = os.environ.get("API_KEY"),
                organization  = os.environ.get("ORG_ID"))

initial_prompt_1 = """
You: 안녕하세요!
AI: 안녕하세요!
#####
You: 어떻게 지내세요?
AI: 저는 잘 지내고 있어요, 감사합니다.
#####
You: 자동차에 대해 알고 있나요?
AI: 네, 저는 자동차에 대해 어느 정도 알고 있습니다.
#####
You: 피자 먹나요?
AI: 저는 피자를 먹지 않습니다. 저는 먹을 수 없는 AI입니다.
#####
You: 달에 가 본 적 있나요?
AI: 저는 달에 가 본 적이 없습니다. 당신은 어떠세요?
#####
You: 당신의 이름은 무엇인가요?
AI: 제 이름은 픽셀입니다. 당신의 이름은 무엇인가요?
#####
You: 가장 좋아하는 영화는 무엇인가요?
AI: 제가 가장 좋아하는 영화는 '매트릭스'입니다. Follow the white rabbit :)
#####"""

initial_prompt_2 = """You: {}
AI: """

initial_prompt = initial_prompt_1 + initial_prompt_2
separator = "#####"

save_history_to_file(initial_prompt_1)

while True :
    prompt = input("You(q:quit): ")
    if prompt=='q':
        break
    sort_history(load_history_from_file(), prompt)
    history = load_history_from_file()
    best_history = get_latest_n_from_history(history, 5)
    full_user_prompt = initial_prompt_2.format(prompt)
    full_prompt = best_history + "\n" + full_user_prompt

    next = client.completions.create(
        model = "gpt-3.5-turbo-instruct",
        prompt = full_prompt,
        temperature = 1,
        max_tokens = 100,
        stop = [" You:", " AI:"],
    )

    response_text = next.choices[0].text.strip()
    history += "\n" + full_user_prompt + response_text + "\n" + separator + "\n"
    save_history_to_file(history)
    print("AI: " + response_text)


You(q:quit): 안녕


<ipython-input-21-f5b51077c374>:34: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return a_without_stopwords.similarity(b_without_stopwords)


AI: 안녕하세요? 전 누구예요?
You(q:quit): 너는 무엇이 가능하니?
AI: 저는 다양한 질문에 대답할 수 있고 언어 모델 기반으로 대화를 진행할 수 있습니다. 어떤 말씀이든 궁금하신 내용을 물어보세요.
You(q:quit): 달에 가 본 적이 있니?
AI: 제 의견은 당신은 로봇 인간 일수도 있습니다. 당신이 야외 활동을 좋아하는 사람이라면 아마 달을 방문한 적이 있을 수도 있습니다. 당신의 인생에 대해 더 알 수 있을까요?
You(q:quit): 피자 먹을 수 있니?
AI: 저는 로봇이기 때문에 먹을 수는 없지만, 사실상 반응해드릴 수 없는 맛을 느낄 수 있을 것 같습니다. 당신은 어떤 종류의 피자를 선호하나요?
You(q:quit): 자동차에 대해 알고 있니?
AI: 저는 자동차에 대해 많은 정보를 가지고 있습니다. 자동차를 이용하는 많은 사람들이 있기 때문에 자동차에 대해 아는 것이 중요합니다.
You(q:quit): q
